In [1]:
days = 20*5

DemandC4 = [1800 1400 1700 1400 1600 0]

Mfactor = 60;
C41factor = 4;

InvC4 = zeros(days+50,1)
InvM = zeros(days+50,1)
InvC41 = zeros(days+50,1)

InvM[1:end] = 20;
InvC41[1:end] = 40;

cycleC4 = 1664

t=1;
demandMonth = 1
nOrders=0
noOrder = true
while t < days
    if t >= (demandMonth*20)
        InvC4[t:end] -= DemandC4[demandMonth]
        demandMonth += 1
    end
    if InvC4[t] > DemandC4[demandMonth]
        keepProduction = false
    else
        keepProduction = true
    end

    Mleft = (DemandC4[demandMonth] - InvC4[t]) * Mfactor
    C41left = (DemandC4[demandMonth] - InvC4[t]) * C41factor
    if ((InvM[t] < Mleft) || (InvC41[t] < C41left)) && noOrder && keepProduction
        nOrders+=1
        tLast = t
        tAdd = round(Int64,2*randn(1)[1] + 5)
        if tAdd < 0
            t+=0
        else
            t+=tAdd
        end
        
        InvM[tLast:t] = InvM[tLast]
        InvC41[tLast:t] = InvC41[tLast]
        InvC4[tLast:t] = InvC4[tLast]
        
        InvM[t:end] += max(DemandC4[demandMonth]* Mfactor, cycleC4 * Mfactor * 7)
        InvC41[t:end] += max(DemandC4[demandMonth]* C41factor, cycleC4 * C41factor * 7)
    else
        if InvC4[t] > DemandC4[demandMonth]
            keepProduction = false
        else
            keepProduction = true
        end

        while ((InvM[t]-Mfactor)>0) && ((InvC41[t]-C41factor)>0) && ((t+1)<= days) && keepProduction          
            prod = round(Int64, 200*randn(1)[1] + cycleC4/2)
            nM = div(InvM[t], Mfactor)
            nC41 = div(InvC41[t], C41factor)
            
            nC4 = min(nM, nC41, prod)
            
            InvC4[t:end] += nC4
            InvM[t:end] -= nC4*Mfactor
            InvC41[t:end] -= nC4*C41factor            
            defect = rand()/10
            
            InvC4[t:end] -= round(Int64, InvC4[t+1]*defect)
            if InvC4[t] > DemandC4[demandMonth]
                break
            end
            t+=1
            if t >= (demandMonth*20)
                InvC4[t:end] -= DemandC4[demandMonth]
                demandMonth += 1
            end
            Mleft = (DemandC4[demandMonth] - InvC4[t]) * Mfactor
            C41left = (DemandC4[demandMonth] - InvC4[t]) * C41factor
            if ((InvM[t] < Mleft) || (InvC41[t] < C41left)) && noOrder
                noOrder = false
                nOrders += 1
                tLast = t
                tAdd = round(Int64,2*randn(1)[1] + 5)
                if tAdd < 0
                    tOrder=t
                else
                    tOrder=tAdd+t
                end

                InvM[tLast:tOrder] = InvM[tLast]
                InvC41[tLast:tOrder] = InvC41[tLast]
                InvC4[tLast:tOrder] = InvC4[tLast]
            end
            if !noOrder
                if !(((InvM[t]-Mfactor)>0) && ((InvC41[t]-C41factor)>0) && ((t+1)<= days) && keepProduction)
                    t = tOrder
                    noOrder = true
                    InvM[t:end] += max(Mleft* Mfactor, cycleC4 * Mfactor * 7)
                    InvC41[t:end] += max(C41left* C41factor, cycleC4 * C41factor * 7)
                end
                
            end
        end
    end
    t+=1
end

nOrders

1

In [2]:
using Plots
plot(1:(days+50), InvM/Mfactor, label = "M scaled",title="Inventory over 5 months")
plot!(1:(days+50), InvC41/C41factor, label="C41 scaled")
plot!(1:(days+50), InvC4, label="C4 actual")

using Images
using FileIO
savefig("c4invall.png")

[Plots.jl] Initializing backend: pyplot


In [3]:
plot(1:(days+50), InvC4, label="C4", title="Inventory of C4")

#using Images
#using FileIO
savefig("c4inv.png")